# Conversion class

In [7]:
import pandas as pd
import numpy as np
import sys

class eco_2_5_convert:
    def __init__(self, model_file:str, model_name:str,
                EI_3_8 = False):
        """ALLOWED_UINTS: ['cubic meter', 'cubic meter-year', 'EUR', 'EUR2005', 'guest night',
       'hectare', 'hour', 'kg*day', 'kilo Becquerel', 'kilogram',
       'kilometer', 'kilometer-year', 'kilowatt hour', 'litre',
       'megajoule', 'meter', 'meter-year', 'person kilometer', 'Sm3',
       'square meter', 'square meter-year', 'ton kilometer', 'unit']"""
        
        """model_file: the excel model file path
        model_name: a name provided by the user to name the output file and name the model activity
        info_tab: the "info" tab in the eco 2.5 template is a tab where you can add whatever you want
        so it can be used to add calculations and assumptions"""
        
#         
        #Read database ids files
        #ps: make sure all the files are in the same directory you're running jupyter.
        self.activities_raw_ids = pd.read_csv('Activities_id,_Activ_1686047418567_3_9.csv')
        self.products = pd.read_csv('product_3_9.csv')
        self.categories_default = pd.read_csv('Categories_Latest.csv')
        self.units =  pd.read_csv('Units_updated.csv')
        #Read our LCA custom model
        self.model_activity = pd.read_excel(model_file, sheet_name = 'activity')
        self.model_activity['product_type'] = 'material'
        self.model_exchanges = pd.read_excel(model_file, sheet_name = 'exchange')
        self.model_name = model_name
        self.EI_3_8 = EI_3_8
        self.activity_uom = self.model_activity['uom'].iloc[0]
#         activities_products = self.products.merge(self.activities_raw_ids, on = ['products_id','products_raw_name'])

    def convert(self):
        #merge ids with our model to extract all needed ids for the new template
        
        bio_exchange = self.model_exchanges[self.model_exchanges['database'] == 'biosphere3']
        ei_exchange = self.model_exchanges[self.model_exchanges['database'] == 'ecoinvent 3.9']


        bio_exchange_merge = bio_exchange.merge(self.activities_raw_ids, left_on = 'process name',
                                                          right_on = 'activities_name')
        ei_exchange_merge = ei_exchange.merge(self.activities_raw_ids,
                                             left_on = ['product name', 'process name', 'location name'],
                 right_on = ['products_raw_name', 'activities_name', 'locations_raw_name'])
        exchanges_merge = pd.concat([bio_exchange_merge, ei_exchange_merge])

        
#         exchanges_merge = self.model_exchanges.merge(self.activities_raw_ids, left_on = ['product name', 'process name', 'location name'],
#                  right_on = ['products_raw_name', 'activities_name', 'locations_raw_name'])
        self.model_activity['raw_uom'] = self.model_activity['uom'].str.lower().str.strip()
        unit_merge = self.model_activity.merge(self.units, left_on = 'raw_uom',
                                              right_on = 'units_raw_name')
        if self.activity_uom not in self.units['units_raw_name'].unique():
            print(f"Activity uom is not of the allowed units please change activity uom to: {units['units_raw_name'].unique()}")
            print("Conversion Aborted")
            sys.exit()
        
        location_merge = self.activities_raw_ids.merge(self.model_activity, left_on = 'locations_raw_name',
                                                       right_on = 'location name')
        new_activity  = pd.read_excel('eco 2.5 model upload template.xlsx',
                                      sheet_name = 'activity')
        if 'transport' in self.model_name:
            category_name = 'transport'
        elif 'transport' in self.model_name and 'passenger' in self.model_name:
            category_name = 'transport of passengers'
        else:

            category_name = self.model_activity['category name'].iloc[0]
        
        category_id = self.categories_default[self.categories_default['categories_raw_name']==category_name]['categories_id'].iloc[0]
        
        new_activity['Value'] = np.nan
        
        new_activity = new_activity.set_index(['Column','Table'])
        
        new_activity.loc['id', 'source'] = '5b1acea4-a550-11ed-a811-65431102791f'
        
        new_activity.loc['name', 'activity'] = self.model_name
        
        new_activity.loc['unit id', 'activity'] = unit_merge['units_id'].iloc[0]
        
        new_activity.loc['unit name', 'activity'] = unit_merge['units_raw_name'].iloc[0]
        
        new_activity.loc['amount', 'activity'] = unit_merge['amount'].iloc[0]
        
        new_activity.loc['location id', 'location'] = location_merge['activities_location_id'].iloc[0]
        
        new_activity.loc['location name', 'location'] = location_merge['locations_raw_name'].iloc[0]
        
        new_activity.loc['name', 'product'] = self.model_activity['product name'].iloc[0]
        
        new_activity.loc['type', 'product'] = self.model_activity['product_type'].iloc[0]
        
        new_activity.loc['name', 'process'] = self.model_activity['process name'].iloc[0]
        
        new_activity.loc['category id', 'product'] = category_id

        new_activity = new_activity.reset_index()
        
            
        type_map = {'biosphere3':'biosphere',
                   'ecoinvent 3.9':'technosphere',
                   'ecoinvent 3.8':'technosphere'}
        exchanges_merge['type'] = exchanges_merge['database'].str.strip().map(type_map) 

        exchanges = exchanges_merge[['activities_id', 'amount', 'products_raw_name', 'processes_raw_name', 'type']].drop_duplicates()
        exchanges = exchanges.rename(columns = {'products_raw_name':'product name',
                                           'processes_raw_name':'process name',
                                               'activities_id':'activity id'})
        
        
        
        writer = pd.ExcelWriter('results/ECO_2_5_{}_model.xlsx'.format(self.model_name), engine='xlsxwriter')
        
        new_activity.to_excel(writer, sheet_name='activity', index = False)
        
        exchanges.to_excel(writer, sheet_name='exchanges', index = False)
        
        writer.close()
        
        return new_activity, exchanges

In [8]:
# %%time
#call class with inputs
#eco_25 = eco_2_5_convert('/home/mariam/bw2_shared/notebooks/Furniture_Production_Wooden_TW.xlsx', 'Furniture, wooden, TW')

#get outputs from conversion functions
#new_activity, exchanges = eco_25.convert()

In [9]:
import os
missing_files = []
folder_path = '/Users/burcugulsen/Desktop/transport39'
counter = 0
files = os.listdir(folder_path)
for file in files:
    try:
        
        model_name = file.replace('.xlsx','')
        full_paths = os.path.join(folder_path, file)
        eco_25 = eco_2_5_convert(full_paths,model_name )
        new_activity, exchanges = eco_25.convert()
    except:
        missing_files.append(full_paths)

In [42]:
missing_files 

['/Users/burcugulsen/Desktop/tra/.DS_Store',
 '/Users/burcugulsen/Desktop/tra/product_3_9.csv',
 '/Users/burcugulsen/Desktop/tra/eco 2.5 model upload template.xlsx',
 '/Users/burcugulsen/Desktop/tra/Activities_id,_Activ_1686047418567_3_9.csv',
 '/Users/burcugulsen/Desktop/tra/results',
 '/Users/burcugulsen/Desktop/tra/Units_updated.csv',
 '/Users/burcugulsen/Desktop/tra/.ipynb_checkpoints',
 '/Users/burcugulsen/Desktop/tra/Categories_Latest.csv',
 '/Users/burcugulsen/Desktop/tra/Conversion_script_transport_3.9.ipynb']